## Introduction to ANN - Exercise
---
Instructions are given in <span style="color:blue">blue</span> color.

Here, there really is just one task: Repeat the training part of the notebook we used in class replacing the MNIST dataset with [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html), but also do some experimentation with the MLP-Parameters:
* $n_h$: number of hidden layers,
* $m_j,\>j=1,\dots,n_h$: number of neurons in the $j$-th hidden layer,
* and other hyperparameters that have an impact on performance and training time.

### CIFAR-10

The CIFAR-10 dataset consists of $32 \times 32 \times 3$ RGB pixel images, where each pixel is represented by a RGB value. The third dimension (or *channel* in the case of image data) represents the color value. Therefore, we have $32\times32\times3=3072$ *features* per row. The images are then *unrolled* (or flattened) into vectors of length $3072$ and each one (in both the training set and the test set) is given a *class label* of $0-9$ corresponding to the type of class it represents (where the number corresponds to the alphabetic ordering of the classes).

You fill find the dataset publicly available on [Alex Krizhevsky's Homepage](https://www.cs.toronto.edu/~kriz/cifar.html). To load the data you can use an *adaptation* of the following code from [Matt Petersen](https://mattpetersen.github.io/load-cifar10-with-numpy). Either specify a path and the data will automatically be downloaded, or download the binary version of the data beforehand.
This is done with the following code along with some restructuring in order to be able to work with the data in very much the same way we did with MNIST in class.

**Note**: It might actually take a while to execute the next couple of code cells, especially if the data needs to be downloaded first. Just be patient.

In [ ]:
"""Load from /home/USER/data/cifar10 or elsewhere; download if missing."""

import tarfile
import os
from urllib.request import urlretrieve
import numpy as np


def cifar10(path=None):
    r"""Return (train_images, train_labels, test_images, test_labels).

    Args:
        path (str): Directory containing CIFAR-10. Default is
            /home/USER/data/cifar10 or C:\Users\USER\data\cifar10.
            Create if nonexistant. Download CIFAR-10 if missing.

    Returns:
        Tuple of (train_images, train_labels, test_images, test_labels), each
            a matrix. Rows are examples. Columns of images are pixel values,
            with the order (red -> blue -> green). Columns of labels are a
            onehot encoding of the correct class.
    """
    url = 'https://www.cs.toronto.edu/~kriz/'
    tar = 'cifar-10-binary.tar.gz'
    files = ['cifar-10-batches-bin/data_batch_1.bin',
             'cifar-10-batches-bin/data_batch_2.bin',
             'cifar-10-batches-bin/data_batch_3.bin',
             'cifar-10-batches-bin/data_batch_4.bin',
             'cifar-10-batches-bin/data_batch_5.bin',
             'cifar-10-batches-bin/test_batch.bin']

    if path is None:
        # Set path to /home/USER/data/cifar10 or C:\Users\USER\data\cifar10
        path = os.path.join(os.path.expanduser('~'), 'data', 'cifar10')

    # Create path if it doesn't exist
    os.makedirs(path, exist_ok=True)

    # Download tarfile if missing
    if tar not in os.listdir(path):
        urlretrieve(''.join((url, tar)), os.path.join(path, tar))
        print("Downloaded %s to %s" % (tar, path))

    # Load data from tarfile
    with tarfile.open(os.path.join(path, tar)) as tar_object:
        # Each file contains 10,000 color images and 10,000 labels
        fsize = 10000 * (32 * 32 * 3) + 10000

        # There are 6 files (5 train and 1 test)
        buffr = np.zeros(fsize * 6, dtype='uint8')

        # Get members of tar corresponding to data files
        # -- The tar contains README's and other extraneous stuff
        members = [file for file in tar_object if file.name in files]

        # Sort those members by name
        # -- Ensures we load train data in the proper order
        # -- Ensures that test data is the last file in the list
        members.sort(key=lambda member: member.name)

        # Extract data from members
        for i, member in enumerate(members):
            # Get member as a file object
            f = tar_object.extractfile(member)
            # Read bytes from that file object into buffr
            buffr[i * fsize:(i + 1) * fsize] = np.frombuffer(f.read(), 'B')

    # Parse data from buffer
    # -- Examples are in chunks of 3,073 bytes
    # -- First byte of each chunk is the label
    # -- Next 32 * 32 * 3 = 3,072 bytes are its corresponding image

    # Labels are the first byte of every chunk
    labels = buffr[::3073]

    # Pixels are everything remaining after we delete the labels
    pixels = np.delete(buffr, np.arange(0, buffr.size, 3073))
    images = pixels.reshape(-1, 3072).astype('float32') / 255

    # Split into train and test
    train_images, test_images = images[:50000], images[50000:]
    train_labels, test_labels = labels[:50000], labels[50000:]

    return train_images, train_labels, test_images, test_labels

In [ ]:
X_train, y_train, X_test, y_test = cifar10('./data')
print(f'Train - Rows: {X_train.shape[0]}, columns: {X_train.shape[1]}')
print(f'Test - Rows: {X_test.shape[0]}, columns: {X_test.shape[1]}')

The CIFAR-10 data has a different structure than the MNIST data. To get the same structure as in the class example we need to switch some dimensions and reshape the array.

In [ ]:
X_train = (np.moveaxis(X_train.reshape(50000,3,32,32),1,-1)).reshape(50000,3072)
X_test = (np.moveaxis(X_test.reshape(10000,3,32,32),1,-1)).reshape(10000,3072)

In [ ]:
X_train.shape[1]

To remove the color channel from our images we have to reshape a little differently.

In [ ]:
# Calculating the mean of all 3 channels as the greyscale value:
X_train_g = np.mean(X_train.reshape(50000,32,32,3),-1).reshape(50000,1024)
X_test_g = np.mean(X_test.reshape(10000,32,32,3),-1).reshape(10000,1024)

In [ ]:
X_train_g.shape[1]

* <div style="color:blue">Let's get started with a little visualization task by displaying the 42nd image of each class present.</div>

**Hint**: Remember that you are working with an additional color dimension!

In [ ]:
# Your solution goes here:


* <div style="color:blue">Visualize 25 different samples of the class representing dogs.</div>

In [ ]:
# Your solution goes here:


You are now asked to perform image classification using a multi-layer perceptron:

* <div style="color:blue">Use <code>scikit learn</code>'s <code>MLPClassifier</code> to train various models that specify</div>

    * <div style="color:blue">different numbers of hidden layers,</div>
    * <div style="color:blue">different numbers of neurons in the hidden layers,</div>
    * <div style="color:blue">different hyperparameters (see info below).</div>
* <div style="color:blue">For each of your model variants you should record</div>

    * <div style="color:blue">the time it took to train the network,</div>
    * <div style="color:blue">how many epochs were necessary,</div>
    * <div style="color:blue">how the loss values evolved over the epochs,</div>
    * <div style="color:blue">the accuracy of both training and test data.</div>
* <div style="color:blue">What is your <b>interpretation</b> of those experiments? Give a comment on the behaviour of the different MLP configurations.</div>

**Hints**: 
* The CIFAR-10 data is much more complex than MNIST (as seen in class). The image resolution is very small and the classes are much more *nuanced*. In order to successfully classify such data, you might need to tune some parameters of your model.
* Due to the bigger input size, training times might be much slower than you have seen in class. You absolutely should change some hyperparameters that have an impact on the training times (if you don't want to wait for hours). Here are some **ideas** you could try: 
    + `batch_size`: Larger batch sizes mean less gradient updates are computed, but calculating the gradient is more computationally expensive. So there is an inherent trade-off in the magnitude of batch sizes. Not only that, but different batch sizes will have an impact on model performance. Increasing the batch size might lead to a solution that is closer to the true global minimum (which can actually be undesired as the model loses its ability to generalize). On the other hand, decreasing the batch size might get the model stuck at a bad local minimum early on in the training process. For further information see [Goodfellow et al., 2016](http://www.deeplearningbook.org/contents/optimization.html).
    + `learning_rate`: Higher learning rates will reduce the model's convergence time, but can result in suboptimal weight updates towards the end of the learning process (which can lead to increased bias of the model).
    + `hidden_layer_sizes`: Reducing the number of layers and neurons will result in a less complex model and, therefore, faster training times. However, if the model is too simple, it might not be able to fit the data at hand, which results in low accuracy.
    + `max_iter`: You can force an early stop of the training process with this parameter. This is actually a valid strategy to prevent overfitting, but the model might not fit the data yet if you stop too early. Don't be shy to try really low values!
    + **Data Transformation**: Sometimes it can make sense to transform the data to improve training time. You can always train with less data of course, but this might lead to poor performance of the model. For the image classification task, you can also try to train with *greyscale* only, which would reduce input size by 2/3. Maybe the model is mostly invariant to the *color component*?
<br>
<br>
* If you want to see some more information during the training process, you can turn on the `verbose` parameter (recommended!).
* Getting *above* 50% Test-Accuracy in a short amount of training time would be a really good result! Can you do it?

In [ ]:
# Your solution goes here: (if you need more code/markdown cells, feel free to insert them as you see fit)


* <div style="color:blue">Visualize 12 misclassifications <b>and</b> state an opinion on these.</div>

In [ ]:
# Your solution goes here: (if you need more code/markdown cells, feel free to insert them as you see fit)
